In [1]:
import gym
#mario import
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Activation, Flatten
from keras.optimizers import Adam
import tensorflow as tf
from collections import deque
import time
import random
from tqdm import tqdm
import cv2
import os
import atari_py

Using TensorFlow backend.


In [2]:
if not os.path.isdir('models'):
    os.makedirs('models')
    
#env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = gym.make('AirRaid-v0')
#env = JoypadSpace(env, SIMPLE_MOVEMENT)
episodes = 100
discount = .99
replay_memory_size = 5000
min_replay_size = 4999
batch_size = 128
update_target_every = 5
show_preview = True
show_every = 5

epsilon = 1
epsilon_decay = .99
min_epsilon = .001


class MarioAgent:
    def __init__(self):
        #Main model
        self.model = self.create_model()

        self.target_model = self.create_model()

        self.target_model.set_weights(self.model.get_weights())

        self.replay_memory = deque(maxlen = replay_memory_size)	# This is replay memory

        self.target_update_counter = 0

    def create_model(self):
        # Cretes a model
        model = Sequential()
        model.add(Conv2D(256, (3, 3), input_shape=(env.observation_space.shape)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(2, 2))
        model.add(Dropout(0.2))

        model.add(Conv2D(256, (3, 3)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(2, 2))
        model.add(Dropout(0.2))

        model.add(Flatten())
        model.add(Dense(64))
        model.add(Activation("relu"))

        model.add(Dense(env.action_space.n, activation="linear"))
        model.compile(loss="mse", optimizer=Adam(lr=0.001), metrics=["accuracy"])

        return model

    def update_replay_memory(self, transition):
      #Adding to replay memory
      # [obersvation_space, action, reward, new_observation_space,done ]
        self.replay_memory.append(transition)
      
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(-1, *state.shape) / 255)[0]
    
    def train(self,terminal_state,step):
      
        if len(self.replay_memory) < min_replay_size:
            return
      
        replay_memory_fraction  = random.sample(self.replay_memory,batch_size)
      
        current_states = np.array([transition[0] for transition in replay_memory_fraction]) / 255
      
        current_qs_list = self.model.predict(current_states)
      
        new_current_states = np.array([transition[3] for transition in replay_memory_fraction]) / 255
      
        future_qs_list = self.target_model.predict(new_current_states)
      
        #features
        X = []
        #label and target
        y = []
      
        for index, (current_state,action,reward,new_current_state, done) in enumerate(replay_memory_fraction):
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + discount * max_future_q
            else:
                new_q = reward
          
        current_qs  = current_qs_list[index]
        current_qs[action] = new_q
        
        X.append(current_state)
        y.append(current_qs)
        
        self.model.fit(np.array(X)/255, np.array(y), batch_size = batch_size, verbose = 0, shuffle = False) #might need terminal_state
      
        if terminal_state:
            self.target_update_counter += 1
        
        if self.target_update_counter > update_target_every:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0
        
agent = MarioAgent()
      
for episode in tqdm(range(1,episodes+1),ascii=True, unit='episodes'):
    episode_reward = 0
    counter = 0
    step = 1
    current_state = env.reset()
    #current_state = cv2.cvtColor(current_state, cv2.COLOR_BGR2GRAY)
  
    done = False
    while not done:
        if np.random.random() > epsilon:
            action = np.argmax(agent.get_qs(current_state))
        else:
            action = np.random.randint(0,env.action_space.n)
            
        new_state, reward, done, info = env.step(action)
        
        episode_reward += reward
    
        #if show_preview and not episode % show_every:
        #if counter % show_every:
        env.render()
        #if len(agent.replay_memory) == (replay_memory_size-1):
        #    agent.replay_memory.popleft()
        print(len(agent.replay_memory))
        agent.update_replay_memory((current_state,action,reward,new_state,done))
        #print("Current State", current_state)
        #print("New State", new_state)
        #if np.array_equal(current_state,new_state):
        #    print("yes")
        #else:
        #    print("no")
        agent.train(done,step)
        #if np.array_equal(agent.get_qs(current_state),agent.get_qs(new_state)):
        #    print('TRUE')
        #else:
        #    print('FALSE')
        current_state = new_state
        step+=1
        if episode % 20 == 0:
            agent.model.save(f'models/{episode}_reward{episode_reward}.model')
        
    if epsilon > min_epsilon:
        epsilon *= epsilon_decay
        epsilon = max(min_epsilon,epsilon)
        
    counter += 1
env.close()

ValueError: Input 0 is incompatible with layer conv2d_1: expected ndim=4, found ndim=2